# Generate Verilog code for the Simplified sparse Ising machine.

In [14]:
"""
Python code to generate an arbitrarily large simple sparse Ising machine factorizer.
 
Details: 
- Unbiased noise for I = 0 cases
- Biased noise (AND unbiased bits) for I = +/-1 cases
- PBits separated into 5 colors according to Greedy graph coloring algorithm.
- 6 PLL generated clocks, phase shifted equal amounts (60 deg.)

Author: Andrew Rockovich	Date: 3/12/25
"""

import numpy as np

#Greedy coloring scheme adapted from the example on Geeksforgeeks.com by mohit kumar
#  https://www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/
def addEdge(adj, v, w):
     
    adj[v].append(w)
     
    # Note: the graph is undirected
    adj[w].append(v)  
    return adj
 
# Assigns colors (starting from 0) to all
# vertices and prints the assignment of colors
def greedyColoring(adj, V):
     
    result = [-1] * V
 
    # Assign the first color to first vertex
    result[0] = 0;
 
 
    # A temporary array to store the available colors. 
    # True value of available[cr] would mean that the
    # color cr is assigned to one of its adjacent vertices
    available = [False] * V
 
    # Assign colors to remaining V-1 vertices
    for u in range(1, V):
         
        # Process all adjacent vertices and
        # flag their colors as unavailable
        for i in adj[u]:
            if (result[i] != -1):
                available[result[i]] = True
 
        # Find the first available color
        cr = 0
        while cr < V:
            if (available[cr] == False):
                break
             
            cr += 1
             
        # Assign the found color
        result[u] = cr 
 
        # Reset the values back to false 
        # for the next iteration
        for i in adj[u]:
            if (result[i] != -1):
                available[result[i]] = False
                
    return result

from gen_sparse_j_h_verilog import *

def Gen_Verilog_multiplier(N,k,product,file,name):
    """
    Inputs: N (NxN multiplier, int), k (maximum nearest neighbors, int), 
            file (where to write to, .v file, string),
            module (module name, string)
    
    """
    #===== Define lookup table and other parts of multiplier: ====
    I_lookup, nonzero_indices, size, outs, zeros = I_table(N,k)
    J,h, size, outs, zeros = J_mult(N,k)
    print(I_lookup[-1])
    #Greedy graph coloring on Multiplier J matrix:
    g = [[] for i in range(size)]
    for i in range(size):
        for j in range(size):
            if(J[i][j]!=0):
                g = addEdge(g,i,j)
    colors = greedyColoring(g,size)
    print(colors)
    #Organize by color so we know what to update at each clock
    biasCount = np.zeros(5,dtype=int)
    unbiasCount = np.zeros(5,dtype=int)
    RNGperColor = np.zeros(5,dtype=int)
    for i in range(size):
        oneFlag = 0
        zeroFlag = 0
        if ((i not in outs) and (i not in zeros)):
            for j in range(len(I_lookup[i])):
                if (I_lookup[i][j][-1]==0):
                    zeroFlag = 1
                elif (abs(I_lookup[i][j][-1])==1):
                    oneFlag = 1
            if (oneFlag == 1):
                biasCount[colors[i]] += 1
            if (zeroFlag == 1):
                unbiasCount[colors[i]] += 1
            if (oneFlag | zeroFlag):
                RNGperColor[colors[i]] += 1
    
    LFSRperColor = np.zeros(5,dtype=int)
    for i in range(5):
        LFSRperColor[i] = np.ceil(((biasCount[i]*3)+unbiasCount[i])/46)
    
    print("in order, there are ",biasCount[0]+unbiasCount[0],", ",biasCount[1]+unbiasCount[1],", ",biasCount[2]+unbiasCount[2],", ",biasCount[3]+unbiasCount[3],", ",biasCount[4]+unbiasCount[4]," PBits per color, for ",np.sum(biasCount)+np.sum(unbiasCount)," total.\n")
 
    # Update LFSR and PBits on posedge,  Update randoms on negedge.
    
    #============== Write beginning of file =================          
    test = open(file,'w')
    
    header = [
        "//Generated automatically via 'Gen_VerilogRunTilDone_LFSR_3-25.ipynb python code'\n\n",
        "`timescale 1ns / 1ps\n\n",
        "module ",name,"(\n",
        "    input SYS_CLK_100M_P,\n",
        "    input SYS_CLK_100M_N,\n",
        "    output W_LED_0,\n",
        "    output W_LED_1,\n",
        "    output W_LED_2,\n",
        "    output W_LED_3\n",
        "    );\n\n"]


    variable_defs = [
        "wire sample_clk;\n",
        "wire color0_clk;\n",
        "wire color1_clk;\n",
        "wire color2_clk;\n",
        "wire color3_clk;\n",
        "wire color4_clk;\n",
        "reg [37:0] counter;\n",
        "initial counter = 38'b0;\n",
        f"reg [{int(2*N-1)}:0] solution;\n",
        f"reg [{int(2*N-1)}:0] solution_check;\n",
        f"wire [{int(2*N-1)}:0] solution_set;\n"]
    variable_defs.append(f"initial solution_check = {int(2*N)}'b")
    for i in range(2*N):
        variable_defs.append(str(int(bool((2**(2*N-1-i))&product))))
    variable_defs.append(";\n")
    variable_defs.append("reg solution_flag;\n")
    variable_defs.append("initial solution_flag = 1'b0;\n")
    variable_defs.append("reg failure;\n")
    variable_defs.append("initial failure = 1'b0;\n")
    variable_defs.append(f"reg [0:{int(size-len(outs)-len(zeros)-1)}] InitCond;\n")
    variable_defs.append("reg run;\n")
    variable_defs.append(f"wire [{int(LFSRperColor[0]*46)-1}:0] LFSRcolor0;\n")
    variable_defs.append(f"wire [{int(LFSRperColor[1]*46)-1}:0] LFSRcolor1;\n")
    variable_defs.append(f"wire [{int(LFSRperColor[2]*46)-1}:0] LFSRcolor2;\n")
    variable_defs.append(f"wire [{int(LFSRperColor[3]*46)-1}:0] LFSRcolor3;\n")
    variable_defs.append(f"wire [{int(LFSRperColor[4]*46)-1}:0] LFSRcolor4;\n")
    variable_defs.append(f"reg [{sum(biasCount)-1}:0] BiasedRNG;       //For I=+/-1 cases\n")
    variable_defs.append(f"reg [{sum(unbiasCount)-1}:0] UnbiasedRNG;   //For I=0 cases\n")
    variable_defs.append(f"reg [0:{size-1}] m;\n")
    variable_defs.append("//To keep from synthesizing away:\n")
    variable_defs.append("assign W_LED_0=m[0];\n")
    variable_defs.append("assign W_LED_1=m[1];\n")
    variable_defs.append("assign W_LED_2=failure;\n")
    variable_defs.append("assign W_LED_3=solution_flag;\n\n")
    variable_defs.append("//Initialize the system for Reverse operation:\n")
    for i in range(2*N):
        variable_defs.append(f"initial m[{int(outs[i])}] = 1'b{int(bool(product&(2**i)))};\n")
    variable_defs.append("\n//Initialize the PBits clamped to zero:\n")
    for i in range(len(zeros)):
        variable_defs.append(f"initial m[{int(zeros[i])}] = 1'b0;\n")

    variable_defs.append("\n")
    
    rng = np.random.default_rng(seed=111)
    
    generateRNG = ["//Generate the pseudo-entropy source:\n"]
    for j in range(LFSRperColor[0]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR0_{j}(.LFSRregister(LFSRcolor0[{46*j+45}:{46*j}]),.clk(sample_clk));\n")
    for j in range(LFSRperColor[1]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR1_{j}(.LFSRregister(LFSRcolor1[{46*j+45}:{46*j}]),.clk(color0_clk));\n")
    for j in range(LFSRperColor[2]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR2_{j}(.LFSRregister(LFSRcolor2[{46*j+45}:{46*j}]),.clk(color1_clk));\n")
    for j in range(LFSRperColor[3]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR3_{j}(.LFSRregister(LFSRcolor3[{46*j+45}:{46*j}]),.clk(color2_clk));\n")
    for j in range(LFSRperColor[4]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR4_{j}(.LFSRregister(LFSRcolor4[{46*j+45}:{46*j}]),.clk(color3_clk));\n")

    test.writelines(header)
    test.writelines(variable_defs)
    test.writelines(generateRNG)
    
    #===================Define update rules:===========================
    RunReset = [
        "//To control whether the system runs or resets using VIO and counter:\n",
        "always @(posedge sample_clk) begin\n",
        "    if (reset) begin\n",
        "        run = 1'b0;\n",
        "        counter = 38'b0;\n",
        f"        solution = {int(2*N)}'b0;\n",
        "        failure = 1'b0;\n",
        "        solution_check = solution_set;\n"]
    for i in range(2*N):
        RunReset.append(f"        m[{int(outs[i])}] = solution_set[{i}];\n")
    RunReset.append("    end else if (solution_flag) begin\n")
    RunReset.append("        run = 1'b0;\n")
    RunReset.append("        counter = 38'b0;\n")
    RunReset.append(f"        solution = {int(2*N)}'b0;\n")
    RunReset.append("        failure = 1'b0;\n")
    RunReset.append("    end else if (counter < 38'b11111111111111111111111111111111111111) begin\n")
    Unclamped_index = 0
    timesteps = 0
    while (Unclamped_index<int(size-len(outs)-len(zeros))):
        if (timesteps==0):
            RunReset.append(f"        if (counter == {timesteps+1}) begin\n")
        else:
            RunReset.append(f"        else if (counter == {timesteps+1}) begin\n")
        for i in range(sum(unbiasCount)):
            if (Unclamped_index<int(size-len(outs)-len(zeros))):
                RunReset.append(f"            InitCond[{Unclamped_index}] = UnbiasedRNG[{i}];\n")
                Unclamped_index+=1
        RunReset.append("        end\n")
        timesteps+=1

    RunReset.append(f"        else if (counter=={int(np.ceil(int(size-len(outs)-len(zeros))/sum(unbiasCount))+2)})\n")
    RunReset.append("            run = 1'b1;\n")
    RunReset.append("        counter = counter+38'b1;\n")
    RunReset.append("        solution = {")
    for i in range(N):
        RunReset.append(f"m[{N-1-i}]")
        if (i<N-1):
            RunReset.append(",")
    RunReset.append("}*{")
    for i in range(N):
        RunReset.append(f"m[{2*N-1-i}]")
        if (i<N-1):
            RunReset.append(",")
    RunReset.append("};\n")
    RunReset.append("    end else begin \n")
    RunReset.append("        counter = 38'b0;\n")
    RunReset.append("        failure = 1'b1;\n")
    RunReset.append("        run = 1'b0;\n")
    RunReset.append("    end\n")
    RunReset.append("end\n\n")
    RunReset.append("//To measure on only the last step using ILA:\n")
    RunReset.append("always @(negedge sample_clk) begin\n")
    RunReset.append("    if (solution_flag)\n")
    RunReset.append("        solution_flag = 1'b0;\n")
    RunReset.append(f"    else if ((run & (solution == solution_check)) | failure)\n")
    RunReset.append("        solution_flag = 1'b1;\n")
    RunReset.append("end\n\n")
    
    
    
    test.writelines(RunReset)
    
    unbiasedOffset = 0
    biasedOffset = 0
    
    Unclamped_index = 0
    PBitupdates = ["//Update the outputs by color:\n"]
    
    #Generate the Color-by-color updates:
    for col in range(k):
        PBitupdates.append(f"always @(posedge color{col}_clk) begin\n")
        for i in range(size):
            if (colors[i]==col and (i not in outs) and (i not in zeros)):
                PBitupdates.append(f"    m[{i}] = run?((")
                I0conditions=[]
                Ipos1conditions=[]
                Ineg1conditions=[]
                Oneconditions=[]
                I0flag = 0
                Ineg1flag = 0
                Ipos1flag = 0
                flag1 = 0
                numI0=0
                numIneg1=0
                numIpos1=0
                num1=0
                for j in range(len(I_lookup[i])):
                    #If I = 0:
                    if (I_lookup[i][j][-1]==0):
                        I0flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numI0!=0):
                                    I0conditions.append("|")
                                I0conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                I0conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    I0conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    I0conditions.append(")")
                                else:
                                    print("Error 1 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                I0conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    I0conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    I0conditions.append(")")
                                else:
                                    print("Error 2 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 3 occured in indexing the PBit update rules according to I_lookup")
                            numI0 += 1
                    #if I=-1:
                    elif (I_lookup[i][j][-1]==-1):
                        Ineg1flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numIneg1!=0):
                                    Ineg1conditions.append("|")
                                Ineg1conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Ineg1conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append(")")
                                else:
                                    print("Error 4 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Ineg1conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append(")")
                                else:
                                    print("Error 5 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 6 occured in indexing the PBit update rules according to I_lookup")
                            numIneg1 += 1
                    #If I=+1:
                    elif (I_lookup[i][j][-1]==1):
                        Ipos1flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numIpos1!=0):
                                    Ipos1conditions.append("|")
                                Ipos1conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Ipos1conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append(")")
                                else:
                                    print("Error 7 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Ipos1conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append(")")
                                else:
                                    print("Error 8 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 9 occured in indexing the PBit update rules according to I_lookup")
                            numIpos1 += 1
                    #If I>1:
                    elif (I_lookup[i][j][-1]>1):
                        flag1=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (num1!=0):
                                    Oneconditions.append("|")
                                Oneconditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Oneconditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Oneconditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Oneconditions.append(")")
                                else:
                                    print("Error 10 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Oneconditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Oneconditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Oneconditions.append(")")
                                else:
                                    print("Error 11 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 12 occured in indexing the PBit update rules according to I_lookup")
                            num1 += 1
                before=0
                if (I0flag):
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(I0conditions).replace(',',''))
                    PBitupdates.append(f")&UnbiasedRNG[{unbiasedOffset}])")
                    unbiasedOffset+=1
                    before=1
                if (Ineg1flag):
                    if (before):
                        PBitupdates.append("|(")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Ineg1conditions).replace(',',''))
                    PBitupdates.append(f")&BiasedRNG[{biasedOffset}])")
                    before=1
                if (Ipos1flag):
                    if (before):
                        PBitupdates.append("|(")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Ipos1conditions).replace(',',''))
                    PBitupdates.append(f")&~BiasedRNG[{biasedOffset}])")
                    before=1
                if (Ineg1flag | Ipos1flag):
                    biasedOffset+=1
                if (flag1):
                    if (before):
                        PBitupdates.append("|")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Oneconditions).replace(',',''))
                    PBitupdates.append(")")
                    before=1
                PBitupdates.append(f"):InitCond[{Unclamped_index}];\n")
                Unclamped_index+=1
        PBitupdates.append("end\n\n")
    
    
    #========== Generate the source of random values ===================
    
    #Create an array from 0 -> len(LFSRcolor*color*), draw from it at random, removing values once drawn
    
    RNGupdates = ["//Update the registered value of RNGs one shifted clock before its needed:\n"]
    RNGupdates.append("always @(posedge sample_clk) begin\n")
    biasedOffset = 0
#     print('biasedOffset is ',biasedOffset,"\n")
    unbiasedOffset = 0
    color_nodes = np.arange(int(LFSRperColor[0]*46))
    for i in range(biasCount[0]):
        RNGupdates.append(f"    BiasedRNG[{i}] = (")
        for j in range(3):
            if (j<2):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor0[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor0[{node}]);\n")
    for i in range(unbiasCount[0]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{i}] = LFSRcolor0[{node}];\n")
    biasedOffset+=biasCount[0]
    unbiasedOffset+=unbiasCount[0]
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color0_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[1]*46))
    for i in range(biasCount[1]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(3):
            if (j<2):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor1[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor1[{node}]);\n")
    for i in range(unbiasCount[1]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor1[{node}];\n")
    unbiasedOffset+=unbiasCount[1]
    biasedOffset+=biasCount[1]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color1_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[2]*46))
    for i in range(biasCount[2]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(3):
            if (j<2):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor2[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor2[{node}]);\n")
    for i in range(unbiasCount[2]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor2[{node}];\n")
    unbiasedOffset+=unbiasCount[2]
    biasedOffset+=biasCount[2]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color2_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[3]*46))
    for i in range(biasCount[3]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(3):
            if (j<2):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor3[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor3[{node}]);\n")
    for i in range(unbiasCount[3]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor3[{node}];\n")
    unbiasedOffset+=unbiasCount[3]
    biasedOffset+=biasCount[3]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color3_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[4]*46))
    for i in range(biasCount[4]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(3):
            if (j<2):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor4[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor4[{node}]);\n")
    for i in range(unbiasCount[4]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor4[{node}];\n")
    RNGupdates.append("end\n\n")


    test.writelines(PBitupdates)
    test.writelines(RNGupdates)
    
    #======= Write end of file ===================================
    PLL = ["//Generate the 40MHz shifted clocks:\n",
           "clk_wiz_0 myPLL(.clk_out1(sample_clk),.clk_out2(color0_clk),.clk_out3(color1_clk),.clk_out4(color2_clk),.clk_out5(color3_clk),.clk_out6(color4_clk),.clk_in1_p(SYS_CLK_100M_P),.clk_in1_n(SYS_CLK_100M_N));\n\n"]
    ILA = ["//Generate the ILA for data collection:\n",
           f"ila_0 ILAinst(.clk(sample_clk),.probe0(run),.probe1(solution_flag),.probe2(failure),.probe3(counter[37:0]));\n\n"]
    VIO = ["//Instantiate VIO:\n",
           f"vio_0 VIOinst (.clk(sample_clk),.probe_out0(reset),.probe_out1(solution_set[{int(2*N-1)}:0]));\n\n"]
    end = ["endmodule\n\n"]
    RNGmodule = ["//Module for generating LFSR:\n",
                "module lfsr #(parameter seed = 46'b1) (output reg[45:0] LFSRregister, input clk);\n\n",
                "//Set it to the seed to begin:\n",
                 "initial begin\n",
                 "    LFSRregister = seed;\n",
                 "end\n\n",
                 "//Shift and replace zeroth bit:\n",
                 "always @(negedge clk) begin\n",
                 "    LFSRregister[45:0] = {LFSRregister[44:0],(LFSRregister[45] ^ LFSRregister[39] ^ LFSRregister[38] ^ LFSRregister[37])};\n",
                 "end\n",
                 "endmodule"]


    test.writelines(PLL)
    test.writelines(ILA)
    test.writelines(VIO)
#     test.writelines(VIO)
    test.writelines(end)
    test.writelines(RNGmodule)
    test.close()


In [15]:
Gen_Verilog_multiplier(16,5,3954787153,'testfile.v','main')

# 15x16 using 16x16 hardware:
# Gen_Verilog_multiplier(16,5,2125923721,'testfile.v','main')

# Gen_Verilog_multiplier(15,5,1064578103,'testfile.v','main')
# Gen_Verilog_multiplier(14,5,265167431,'testfile.v','main')
# Gen_Verilog_multiplier(13,5,65884789,'testfile.v','main')
# Gen_Verilog_multiplier(12,5,16524029,'testfile.v','main')
# Gen_Verilog_multiplier(10,5,999919,'testfile.v','main')
# Gen_Verilog_multiplier(8,5,56153,'testfile.v','main')
# Gen_Verilog_multiplier(6,5,3233,'testfile.v','main')
# Gen_Verilog_multiplier(3,5,10,'testfile.v','main')

print("\n\nVerilog file:\n\n")
test = open('testfile.v','r')
print(test.read())
test.close


There are  2128  PBits in the circuit
[[160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175], [176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191], [192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207], [208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223], [224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239], [240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255], [256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271], [272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287], [288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303], [304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319], [320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335], [336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347



Verilog file:


//Generated automatically via 'Gen_VerilogRunTilDone_LFSR_3-25.ipynb python code'

`timescale 1ns / 1ps

module main(
    input SYS_CLK_100M_P,
    input SYS_CLK_100M_N,
    output W_LED_0,
    output W_LED_1,
    output W_LED_2,
    output W_LED_3
    );

wire sample_clk;
wire color0_clk;
wire color1_clk;
wire color2_clk;
wire color3_clk;
wire color4_clk;
reg [37:0] counter;
initial counter = 38'b0;
reg [31:0] solution;
reg [31:0] solution_check;
wire [31:0] solution_set;
initial solution_check = 32'b11101011101110010100001101010001;
reg solution_flag;
initial solution_flag = 1'b0;
reg failure;
initial failure = 1'b0;
reg [0:2079] InitCond;
reg run;
wire [1057:0] LFSRcolor0;
wire [1379:0] LFSRcolor1;
wire [1011:0] LFSRcolor2;
wire [229:0] LFSRcolor3;
wire [735:0] LFSRcolor4;
reg [1133:0] BiasedRNG;       //For I=+/-1 cases
reg [945:0] UnbiasedRNG;   //For I=0 cases
reg [0:2127] m;
//To keep from synthesizing away:
assign W_LED_0=m[0];
assign W_LED_1=m[1];
assign W_LE

<function TextIOWrapper.close()>

# SIZE GUIDE: Take the above and make it as small as possible, remove ILA, VIO.

In [1]:
"""
Python code to generate an arbitrarily large simple sparse Ising machine factorizer.
 
Details: 
- Unbiased noise for I = 0 cases
- Biased noise for I = +/-1 cases
- PBits separated into 5 colors according to Greedy graph coloring algorithm.
- 6 PLL generated clocks, phase shifted equal amounts (60 deg.)


Author: Andrew Rockovich	Date: 3/12/25
"""

import numpy as np

#Greedy coloring scheme adapted from the example on Geeksforgeeks.com by mohit kumar
#  https://www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/
def addEdge(adj, v, w):
     
    adj[v].append(w)
     
    # Note: the graph is undirected
    adj[w].append(v)  
    return adj
 
# Assigns colors (starting from 0) to all
# vertices and prints the assignment of colors
def greedyColoring(adj, V):
     
    result = [-1] * V
 
    # Assign the first color to first vertex
    result[0] = 0;
 
 
    # A temporary array to store the available colors. 
    # True value of available[cr] would mean that the
    # color cr is assigned to one of its adjacent vertices
    available = [False] * V
 
    # Assign colors to remaining V-1 vertices
    for u in range(1, V):
         
        # Process all adjacent vertices and
        # flag their colors as unavailable
        for i in adj[u]:
            if (result[i] != -1):
                available[result[i]] = True
 
        # Find the first available color
        cr = 0
        while cr < V:
            if (available[cr] == False):
                break
             
            cr += 1
             
        # Assign the found color
        result[u] = cr 
 
        # Reset the values back to false 
        # for the next iteration
        for i in adj[u]:
            if (result[i] != -1):
                available[result[i]] = False
                
    return result

from gen_sparse_j_h_verilog import *
def Gen_Verilog_multiplier(N,k,product,file,name):
    """
    Inputs: N (NxN multiplier, int), k (maximum nearest neighbors, int), 
            file (where to write to, .v file, string),
            module (module name, string)
    
    """
    #===== Define lookup table and other parts of multiplier: ====
    I_lookup, nonzero_indices, size, outs, zeros = I_table(N,k)
    J,h, size, outs, zeros = J_mult(N,k)
    
    #Greedy graph coloring on Multiplier J matrix:
    g = [[] for i in range(size)]
    for i in range(size):
        for j in range(size):
            if(J[i][j]!=0):
                g = addEdge(g,i,j)
    colors = greedyColoring(g,size)
    
    #Organize by color so we know what to update at each clock
    biasCount = np.zeros(5,dtype=int)
    unbiasCount = np.zeros(5,dtype=int)
    RNGperColor = np.zeros(5,dtype=int)
    for i in range(size):
        oneFlag = 0
        zeroFlag = 0
        if ((i not in outs) and (i not in zeros)):
            for j in range(len(I_lookup[i])):
                if (I_lookup[i][j][-1]==0):
                    zeroFlag = 1
                elif (abs(I_lookup[i][j][-1])==1):
                    oneFlag = 1
            if (oneFlag == 1):
                biasCount[colors[i]] += 1
            if (zeroFlag == 1):
                unbiasCount[colors[i]] += 1
            if (oneFlag | zeroFlag):
                RNGperColor[colors[i]] += 1
    
    LFSRperColor = np.zeros(5,dtype=int)
    for i in range(5):
        LFSRperColor[i] = np.ceil(((biasCount[i]*4)+unbiasCount[i])/46)
     
    # Update LFSR and PBits on posedge,  Update randoms on negedge.
    
    #============== Write beginning of file =================          
    test = open(file,'w')
    
    header = [
        "//Generated automatically via 'Gen_VerilogRunTilDone_LFSR_3-25.ipynb python code'\n\n",
        "`timescale 1ns / 1ps\n\n",
        "module ",name,"(\n",
        "    input SYS_CLK_100M_P,\n",
        "    input SYS_CLK_100M_N,\n",
        "    output W_LED_0,\n",
        "    output W_LED_1,\n",
        "    output W_LED_2,\n",
        "    output W_LED_3\n",
        "    );\n\n"]


    variable_defs = [
        "wire sample_clk;\n",
        "wire color0_clk;\n",
        "wire color1_clk;\n",
        "wire color2_clk;\n",
        "wire color3_clk;\n",
        "wire color4_clk;\n",
        "reg [31:0] counter;\n",
        "initial counter = 32'b0;\n",
        f"reg [{int(2*N-1)}:0] solution;\n",
        "reg solution_flag;\n",
        "initial solution_flag = 1'b0;\n",
        "reg failure;\n",
        "initial failure = 1'b0;\n",
        f"wire [{int(LFSRperColor[0]*46)-1}:0] LFSRcolor0;\n",
        f"wire [{int(LFSRperColor[1]*46)-1}:0] LFSRcolor1;\n",
        f"wire [{int(LFSRperColor[2]*46)-1}:0] LFSRcolor2;\n",
        f"wire [{int(LFSRperColor[3]*46)-1}:0] LFSRcolor3;\n",
        f"wire [{int(LFSRperColor[4]*46)-1}:0] LFSRcolor4;\n",
        f"reg [{sum(biasCount)-1}:0] BiasedRNG;       //For I=+/-1 cases\n",
        f"reg [{sum(unbiasCount)-1}:0] UnbiasedRNG;   //For I=0 cases\n",
        f"reg [0:{size-1}] m;\n",
        "//To keep from synthesizing away:\n",
        "assign W_LED_0=m[0];\n",
        "assign W_LED_1=m[1];\n",
        "assign W_LED_2=failure;\n",
        "assign W_LED_3=solution_flag;\n\n",
        "//Initialize the system for Reverse operation:\n"]
    for i in range(2*N):
        variable_defs.append(f"initial m[{int(outs[i])}] = 1'b{int(bool(product&(2**i)))};\n")
    variable_defs.append("\n//Initialize the PBits clamped to zero:\n")
    for i in range(len(zeros)):
        variable_defs.append(f"initial m[{int(zeros[i])}] = 1'b0;\n")

    variable_defs.append("\n")
    
    rng = np.random.default_rng(seed=111)
    
    generateRNG = ["//Generate the pseudo-entropy source:\n"]
    for j in range(LFSRperColor[0]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR0_{j}(.LFSRregister(LFSRcolor0[{46*j+45}:{46*j}]),.clk(sample_clk));\n")
    for j in range(LFSRperColor[1]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR1_{j}(.LFSRregister(LFSRcolor1[{46*j+45}:{46*j}]),.clk(color0_clk));\n")
    for j in range(LFSRperColor[2]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR2_{j}(.LFSRregister(LFSRcolor2[{46*j+45}:{46*j}]),.clk(color1_clk));\n")
    for j in range(LFSRperColor[3]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR3_{j}(.LFSRregister(LFSRcolor3[{46*j+45}:{46*j}]),.clk(color2_clk));\n")
    for j in range(LFSRperColor[4]):
        generateRNG.append("lfsr #(.seed(46'b")
        for i in range(46):
            generateRNG.append(f"{rng.integers(0,2)}")
        generateRNG.append(")) ")
        generateRNG.append(f"LFSR4_{j}(.LFSRregister(LFSRcolor4[{46*j+45}:{46*j}]),.clk(color3_clk));\n")

    test.writelines(header)
    test.writelines(variable_defs)
    test.writelines(generateRNG)
    
    
    initcond = [
        "\n//Set the initial state of unclamped m to random bits:\n"
    ]
    for i in range(size):
        if ((i not in zeros) and (i not in outs)):
            initcond.append(f"initial m[{i}] = {rng.integers(0,2)};\n")
            
    test.writelines(initcond)
    
    #===================Define update rules:===========================
    
    solnCheck = ["\n//Check if the factor state matches the product state:\n"]
    solnCheck.append("always @(posedge sample_clk) begin\n")
    solnCheck.append("    solution = {")
    for i in range(N):
        solnCheck.append(f"m[{N-1-i}]")
        if (i<N-1):
            solnCheck.append(",")
    solnCheck.append("}*{")
    for i in range(N):
        solnCheck.append(f"m[{2*N-1-i}]")
        if (i<N-1):
            solnCheck.append(",")
    solnCheck.append("};\n")
    solnCheck.append("end\n\n")
    solnCheck.append("always @(negedge sample_clk) begin\n")
    solnCheck.append(f"    if (solution == {int(2*N)}'b")
    for i in range(2*N):
        solnCheck.append(str(int(bool((2**(2*N-1-i))&product))))
    solnCheck.append(")\n")
    solnCheck.append("        solution_flag = 1'b1;\n")
    solnCheck.append("    else begin\n")
    solnCheck.append("        if (counter==32'b11111111111111111111111111111111) begin\n")
    solnCheck.append("            failure = 1'b1;\n")
    solnCheck.append("        end else\n")
    solnCheck.append("            counter = counter + 32'b1;\n")
    solnCheck.append("    end\n")
    solnCheck.append("end\n\n")
    
    test.writelines(solnCheck)
    
    unbiasedOffset = 0
    biasedOffset = 0
    
    Unclamped_index = 0
    PBitupdates = ["//Update the outputs by color:\n"]
    #For Color-by-color updates:
    for col in range(k):
        PBitupdates.append(f"always @(posedge color{col}_clk) begin\n")
        for i in range(size):
            if (colors[i]==col and (i not in outs) and (i not in zeros)):
                PBitupdates.append(f"    m[{i}] = (")
                I0conditions=[]
                Ipos1conditions=[]
                Ineg1conditions=[]
                Oneconditions=[]
                I0flag = 0
                Ineg1flag = 0
                Ipos1flag = 0
                flag1 = 0
                numI0=0
                numIneg1=0
                numIpos1=0
                num1=0
                for j in range(len(I_lookup[i])):
                    #If I = 0:
                    if (I_lookup[i][j][-1]==0):
                        I0flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numI0!=0):
                                    I0conditions.append("|")
                                I0conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                I0conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    I0conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    I0conditions.append(")")
                                else:
                                    print("Error 1 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                I0conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    I0conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    I0conditions.append(")")
                                else:
                                    print("Error 2 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 3 occured in indexing the PBit update rules according to I_lookup")
                            numI0 += 1
                    #if I=-1:
                    elif (I_lookup[i][j][-1]==-1):
                        Ineg1flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numIneg1!=0):
                                    Ineg1conditions.append("|")
                                Ineg1conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Ineg1conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append(")")
                                else:
                                    print("Error 4 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Ineg1conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ineg1conditions.append(")")
                                else:
                                    print("Error 5 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 6 occured in indexing the PBit update rules according to I_lookup")
                            numIneg1 += 1
                    #If I=+1:
                    elif (I_lookup[i][j][-1]==1):
                        Ipos1flag=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (numIpos1!=0):
                                    Ipos1conditions.append("|")
                                Ipos1conditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Ipos1conditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append(")")
                                else:
                                    print("Error 7 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Ipos1conditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Ipos1conditions.append(")")
                                else:
                                    print("Error 8 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 9 occured in indexing the PBit update rules according to I_lookup")
                            numIpos1 += 1
                    #If I>1:
                    elif (I_lookup[i][j][-1]>1):
                        flag1=1
                        for r in range(len(I_lookup[i][j])-1):
                            if (r==0):
                                if (num1!=0):
                                    Oneconditions.append("|")
                                Oneconditions.append("(")
                            if (int(I_lookup[i][j][r])==0):
                                Oneconditions.append(f"~m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Oneconditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Oneconditions.append(")")
                                else:
                                    print("Error 10 occured in indexing the PBit update rules according to I_lookup")
                            elif (int(I_lookup[i][j][r])==1):
                                Oneconditions.append(f"m[{int(nonzero_indices[i][r])}]")
                                if(r<(len(I_lookup[i][j])-2)):
                                    Oneconditions.append("&")
                                elif (r==(len(I_lookup[i][j])-2)):
                                    Oneconditions.append(")")
                                else:
                                    print("Error 11 occured in indexing the PBit update rules according to I_lookup")
                            else:
                                print("Error 12 occured in indexing the PBit update rules according to I_lookup")
                            num1 += 1
                before=0
                if (I0flag):
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(I0conditions).replace(',',''))
                    PBitupdates.append(f")&UnbiasedRNG[{unbiasedOffset}])")
                    unbiasedOffset+=1
                    before=1
                if (Ineg1flag):
                    if (before):
                        PBitupdates.append("|(")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Ineg1conditions).replace(',',''))
                    PBitupdates.append(f")&BiasedRNG[{biasedOffset}])")
                    before=1
                if (Ipos1flag):
                    if (before):
                        PBitupdates.append("|(")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Ipos1conditions).replace(',',''))
                    PBitupdates.append(f")&~BiasedRNG[{biasedOffset}])")
                    before=1
                if (Ineg1flag | Ipos1flag):
                    biasedOffset+=1
                if (flag1):
                    if (before):
                        PBitupdates.append("|")
                    PBitupdates.append("(")
                    PBitupdates.append(",".join(Oneconditions).replace(',',''))
                    PBitupdates.append(")")
                    before=1
                PBitupdates.append(f";\n")
                Unclamped_index+=1
        PBitupdates.append("end\n\n")
    
    #Create an array from 0 -> len(LFSRcolor*color*), draw from it at random, removing values once drawn
    
    RNGupdates = ["//Update the registered value of RNGs one shifted clock before its needed:\n"]
    RNGupdates.append("always @(posedge sample_clk) begin\n")
    biasedOffset = 0
#     print('biasedOffset is ',biasedOffset,"\n")
    unbiasedOffset = 0
    color_nodes = np.arange(int(LFSRperColor[0]*46))
    for i in range(biasCount[0]):
        RNGupdates.append(f"    BiasedRNG[{i}] = (")
        for j in range(4):
            if (j<3):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor0[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor0[{node}]);\n")
    for i in range(unbiasCount[0]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{i}] = LFSRcolor0[{node}];\n")
    biasedOffset+=biasCount[0]
    unbiasedOffset+=unbiasCount[0]
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color0_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[1]*46))
    for i in range(biasCount[1]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(4):
            if (j<3):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor1[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor1[{node}]);\n")
    for i in range(unbiasCount[1]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor1[{node}];\n")
    unbiasedOffset+=unbiasCount[1]
    biasedOffset+=biasCount[1]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color1_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[2]*46))
    for i in range(biasCount[2]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(4):
            if (j<3):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor2[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor2[{node}]);\n")
    for i in range(unbiasCount[2]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor2[{node}];\n")
    unbiasedOffset+=unbiasCount[2]
    biasedOffset+=biasCount[2]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color2_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[3]*46))
    for i in range(biasCount[3]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(4):
            if (j<3):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor3[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor3[{node}]);\n")
    for i in range(unbiasCount[3]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor3[{node}];\n")
    unbiasedOffset+=unbiasCount[3]
    biasedOffset+=biasCount[3]
#     print('biasedOffset is ',biasedOffset,"\n")
    RNGupdates.append("end\n\n")
    RNGupdates.append("always @(posedge color3_clk) begin\n")
    color_nodes = np.arange(int(LFSRperColor[4]*46))
    for i in range(biasCount[4]):
        RNGupdates.append(f"    BiasedRNG[{biasedOffset+i}] = (")
        for j in range(4):
            if (j<3):
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor4[{node}]&")
            else:
                index = np.random.randint(0,len(color_nodes))
                node = color_nodes[index]
                color_nodes = np.delete(color_nodes,index)
                RNGupdates.append(f"LFSRcolor4[{node}]);\n")
    for i in range(unbiasCount[4]):
        index = np.random.randint(0,len(color_nodes))
        node = color_nodes[index]
        color_nodes = np.delete(color_nodes,index)
        RNGupdates.append(f"    UnbiasedRNG[{unbiasedOffset+i}] = LFSRcolor4[{node}];\n")
    RNGupdates.append("end\n\n")


    test.writelines(PBitupdates)
    test.writelines(RNGupdates)
    
    #======= Write end of file ===================================
    PLL = ["//Generate the 40MHz shifted clocks:\n",
           "clk_wiz_0 myPLL(.clk_out1(sample_clk),.clk_out2(color0_clk),.clk_out3(color1_clk),.clk_out4(color2_clk),.clk_out5(color3_clk),.clk_out6(color4_clk),.clk_in1_p(SYS_CLK_100M_P),.clk_in1_n(SYS_CLK_100M_N));\n\n"]
    end = ["endmodule\n\n"]
    RNGmodule = ["//Module for generating LFSR:\n",
                "module lfsr #(parameter seed = 46'b1) (output reg[45:0] LFSRregister, input clk);\n\n",
                "//Set it to the seed to begin:\n",
                 "initial begin\n",
                 "    LFSRregister = seed;\n",
                 "end\n\n",
                 "//Shift and replace zeroth bit:\n",
                 "always @(negedge clk) begin\n",
                 "    LFSRregister[45:0] = {LFSRregister[44:0],(LFSRregister[45] ^ LFSRregister[39] ^ LFSRregister[38] ^ LFSRregister[37])};\n",
                 "end\n",
                 "endmodule"]


    test.writelines(PLL)
    test.writelines(end)
    test.writelines(RNGmodule)
    test.close()


In [8]:
# Gen_Verilog_multiplier(16,5,3954787153,'testfile.v','main')
# Gen_Verilog_multiplier(15,5,1064578103,'testfile.v','main')
# Gen_Verilog_multiplier(14,5,265167431,'testfile.v','main')
# Gen_Verilog_multiplier(13,5,65884789,'testfile.v','main')
# Gen_Verilog_multiplier(12,5,16524029,'testfile.v','main')
# Gen_Verilog_multiplier(10,5,999919,'testfile.v','main')
# Gen_Verilog_multiplier(8,5,56153,'testfile.v','main')
Gen_Verilog_multiplier(6,5,3233,'testfile.v','main')
# Gen_Verilog_multiplier(3,5,10,'testfile.v','main')

print("\n\nVerilog file:\n\n")
test = open('testfile.v','r')
print(test.read())
test.close

There are  282  PBits in the circuit
[[24, 25, 26, 27, 28, 29], [30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41], [42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53], [54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65], [66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77], [78, 79, 80, 81, 82, 83], [84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95]]
Output indices are at  [ 96. 135. 145. 160. 180. 205. 230. 250. 265. 275. 280. 281.]
There are  282  PBits in the circuit
[[24, 25, 26, 27, 28, 29], [30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41], [42, 43, 44, 45, 46, 47], [48, 49, 50, 51, 52, 53], [54, 55, 56, 57, 58, 59], [60, 61, 62, 63, 64, 65], [66, 67, 68, 69, 70, 71], [72, 73, 74, 75, 76, 77], [78, 79, 80, 81, 82, 83], [84, 85, 86, 87, 88, 89], [90, 91, 92, 93, 94, 95]]
Output indices are at  [ 96. 135. 145. 160. 180. 205. 230. 250. 265. 275. 280. 281.]
Vertex 0  --->  Color 0
Vertex 1  --->  Color 0
Vertex 2  --->  Color 0
Vertex 3  --->  Color 0
Vertex 4  --->  Color 0
Vertex 5  

<function TextIOWrapper.close()>